# Dezero
    Dezero :  < 밑바닥부터 시작하는 딥러닝 3 >의 오리지널 framework
* 미니멀리즘 : 이해하기 쉽게 만들었다.
* 순수 파이썬 : 파이썬만으로 구현 가능
* Define-by-Run 방식 사용 : 딥러닝에서 수행하는 여러 계산을 실행 시점에 '연결'하는 구조 ( 현대적인 프레임워크들의 중요한 공통 기능 )

    총 60step으로 나누어, Dezero 만들어보기!

# Step1_상자로서의 변수

In [1]:
class Variable:
    def __init__(self, data):
        self.data = data

In [2]:
import numpy as np

data = np.array(1.0)
x = Variable(data)
print(x.data)

1.0


x는 데이터 자체가 아니라, 데이터를 담은 상자이다.  
실제 데이터는 Variable의 data에 보관

In [3]:
x.data = np.array(2.0)
print(x.data)

2.0


이처럼, Variable 클래스를 상자로 사용!

In [4]:
# 참고) numpy의 다차원 배열 
x = np.array(1)
print(x.ndim)

x = np.array([1,2])
print(x.ndim)

x = np.array([[1,2],[3,4]])
print(x.ndim)

0
1
2


# Step2_변수를 낳는 함수
    class : Variable과 Function 연계하기
* class Function
    * 기반 클래스로 두기
    * __call__ 메서드 : Variable에서 데이터 찾기, 포장하기
    * forward 메서드 : NotImplementedError 예외 발생   
        -> 상위 클래스를 설계할 때, 하위 클래스에서 반드시 오버라이드하여 상세하게 구현해야 하는 메소드를 명시하고자 할 경우  
        -> forward 호출 시, 이 메서드는 상속하여 구현해야한다는 사실 알려준다.
<br>

* 구체적인 함수는 Function클래스를 상속한 클래스에서 구현 (class Square 등등..)

In [5]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y) # Variable 형태로 되돌린다.
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
class Square(Function): # 입력값을 제곱해주는 클래스
    def forward(self, x):
        return x**2

In [6]:
x = Variable(np.array(10))
f = Square()
y = f(x)

print(type(y))
print(y.data)

<class '__main__.Variable'>
100


# Step3_함수 연결

In [7]:
class Exp(Function):
    def forward(self, x):
        return np.exp(x)
    
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)
print(y.data)

1.648721270700128


합성함수로, y = (e^(x^2))^2 한 결과  
-> Function 클래스의 __call__메소드의 입출력이 Variable 인스턴스로 통일하게 되어 있어, 여러 함수를 연속하여 적용 가능!

# Step4_수치 미분
    수치 미분 : 미세한 차이를 이용하여 함수의 변화량을 구하는 방법 ( ex)h=0.001=1e-4와 같은 매우 작은 값 사용)
* 컴퓨터는 극한을 취할 수 없으니, h를 극한과 비슷한 값으로 대체
-> 오차 발생!
* 근사 오차 줄이는 방법
    * 중앙차분 : f(x-h), f(x+h) 차이 구하기

In [8]:
def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data)/(2*eps)

f = Square()
x = Variable(np.array(2.0))
print(numerical_diff(f,x))

4.000000000004


In [9]:
# 합성함수 미분해보자!
def f(x):
    A = Square()
    B = Exp()
    C = Square()
    return C(B(A(x)))

x = Variable(np.array(0.5))
print(numerical_diff(f,x))

3.2974426293330694


# Step5_역전파 이론
    역전파 이용시, 미분 효율적으로 계산가능, 결괏값의 오차 더 적다.  
미분값을 출력->입력 방향(역방향)으로 전파하면,  
* 한 번의 전파만으로
* 모든 매개변수에 대한 미분 계산 -> 계산 효율적으로 이루어지기 때문에, 역전파 사용  
<br>

역전파 시에는,
* 순전파 시 사용한 데이터가 필요해 먼저 순전파 하기
* 각 함수가 입력 변수의 값을 기억한 후, 역전파 계산!

# Step6_수동 역전파

In [10]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None # grad : 미분값
        
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        self.input = input # 입력변수 기억
        return output
    
    def forward(self, x):
        raise NotImplementedError()
        
    def backward(self, gy): # 미분 계산하는 역전파 메서드 추가
        raise NotImplementedError() 
        
class Square(Function):
    def forward(self,x):
        y = x**2
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = 2*x*gy
        return gx
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gy):
        x = self.input.data
        gx = np.exp(x)*gy
        return gx

In [11]:
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

In [12]:
y.grad = np.array(1.0)
b.grad = C.backward(y.grad)
a.grad = B.backward(b.grad)
x.grad = A.backward(a.grad)
print(x.grad)

3.297442541400256


Step4에서 수치 미분으로 구한 결과와, 역전파를 통해 구한 결과가 거의 일치한다!   

-> 역전파 제대로 구현했을 가능성이 커진다!

# Step7_역전파 자동화
    순전파를 한 번만 해주면, 어떠한 계산이라도 상관없이 역전파가 자동으로 이루어지게 하자!
    재귀 방식 이용 - 하나 앞 변수의 bakward method 호출 (self.creator=None일때까지)
   
* Define-by-Run : 딥러닝에서 수행하는 계산들을 계산 시점에 '연결'하는 방식

In [20]:
import numpy as np

class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    
    def forward(self, x):
        raise NotImplementedError()
    
    def backward(self, gy):
        raise NotImplementedError()
        
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gx):
        x = self.input.data
        gy = 2*x*gx
        return gy
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gx):
        x = self.input.data
        gy = np.exp(x)*gx
        return gy
    
A = Square()
B = Exp()
C = Square()

x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

assert y.creator == C
assert y.creator.input == b
assert y.creator.input.creator == B
assert y.creator.input.creator.input == a
assert y.creator.input.creator.input.creator == A
assert y.creator.input.creator.input.creator.input == x

역전파 하나하나 도전해보자!!     
<br>

x ->(A)-> a ->(B)-> b ->(C)-> y    

1. 함수를 가져온다 (creator)
2. 함수 입력 가져온다 (input)
3. 함수 backward method 추가 (grad)

In [22]:
y.grad = np.array(1.0)
C = y.creator
b = C.input
b.grad = C.backward(y.grad)

B = b.creator
a = B.input
a.grad = B.backward(b.grad)

A = a.creator
x = A.input
x.grad = A.backward(a.grad)

x.grad

3.297442541400256

그렇다면, 이 작업을 하나하나 하지말고, class Variable에 backward method를 추가해 한번에 재귀적으로 역전파 자동화되게 하자!

In [22]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        f = self.creator # 1. 함수 가져오기
        if f is not None:
            x = f.input # 2. 함수 입력 가져오기
            x.grad = f.backward(self.grad) # 3. 함수의 backward method 추가
            x.backward() # 하나 앞 변수의 backward method 호출 (재귀)
            
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output 
    
    def farward(self):
        raise NotImplementedError()
        
    def backward(self):
        raise NotImplementedError()
        
class Square(Function):
    def forward(self, x):
        y = x**2
        return y
    
    def backward(self, gx):
        x = self.input.data
        gy = 2*x*gx
        return gy
    
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    
    def backward(self, gx):
        x = self.input.data
        gy = np.exp(x)*gx
        return gy

    
A = Square()
B = Exp()
C = Square()
    
x = Variable(np.array(0.5))
a = A(x)
b = B(a)
y = C(b)

# 역전파
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


이와 같이, y의 backward method 호출 시, 역전파 자동으로 실행 완료! 
<br>

자동 미분의 기초 완료!

# Step8_재귀에서 반복문으로
    Step7의 Variable 클래스에 backward method 추가하는 대신, 처리 효율을 위해 backward method 구현 방식 변경해보자!
    재귀를 사용한 구현(Step7) -> '반복문'을 사용한 구현(Step8)

In [20]:
import numpy as np
class Variable:
    def __init__(self,data):
        self.data = data
        self.grad = None
        self.creator = None
        
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        funcs = [self.creator] # creator들로 이루어진 funcs 리스트 선언
        
        while funcs:
            f = funcs.pop() # funcs 리스트에서 처리해야 할 함수 호출 /pop : 마지막원소부터 꺼내줌
            x, y = f.input, f.output # 함수의 입력 출력 가져온다
            x.grad = f.backward(y.grad) # backward method 호출
            
            if x.creator is not None:
                funcs.append(x.creator) # 하나 앞의 함수를 리스트에 추가

class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(y)
        output.set_creator(self)
        self.input = input
        self.output = output
        return output
    def forward(self):
        raise NotImplementedError()
    def backward(self):
        raise NotImplementedError()

class Square(Function):
    def forward(self, x):
        y = x**2
        return y 
    def backward(self, gx):
        x = self.input.data
        gy = 2*x*gx
        return gy
class Exp(Function):
    def forward(self, x):
        y = np.exp(x)
        return y
    def backward(self, gx):
        x = self.input.data
        gy = np.exp(x)*gx
        return gy
    
A = Square()
B = Exp()
C = Square()
    
x = Variable(np.array(0.5))

a = A(x)
b = B(a)
y = C(b)

y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


일반적으로, 재귀를 이용한 호출보다 '반복문'을 이용한 호출이 처리 효율이 더 좋다.  

# Step9_함수를 더 편리하게
     Dezero 함수에 세 가지 개선 추가!
     사용자의 번거로움 줄이기 위해!

### 1. 파이썬 함수로 이용하기
Square의 클래스를 사용하는 계산을 하려면,  
    Square의 클래스의 인스턴스를 생성한 후, 이어서 그 인스턴스를 호출해야함. -> 번거로움  
    -> 개선 방법 : Dezero 함수를 '파이썬 함수'로 이용


In [5]:
def square(x):
    return Square()(x) # 한 줄로 표현!

def exp(x):
    return Exp()(x)

x = Variable(np.array(0.5))
y = square(exp(square(x))) # 연속하여 적용시킴
y.grad = np.array(1.0)
y.backward()
print(x.grad)

3.297442541400256


### 2. backward 메서드 간소화
    방금 작성한 코드에서 y.grad = np.array(1.0) 부분 생략해보자!

In [19]:
class Variable:
    def __init__(self, data):
        self.data = data
        self.grad = None
        self.creator = None
    def set_creator(self, func):
        self.creator = func
        
    def backward(self):
        if self.grad is None: # 변수의 grad가 None이면, 자동으로 미분값 생성
            self.grad = np.ones_like(self.data) # self.data와 형상, 데이터 타입이 같은 1로 채워진 ndarray 인스턴스 생성됨. 
        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)
            
            if x.creator is not None:
                funcs.append(x.creator)

x = Variable(np.array(0.5))
y = square(exp(square(x)))
y.backward()
print(x.grad)

3.297442541400256


### 3. ndarray만 취급하기
Variable의 데이터로 ndarray 인스턴스만 취급하게끔 의도했다.  
잘못된 데이터 타입 들어오면 오류 발생시키기.

In [2]:
class Variable:
    def __init__(self,data):
        if data is not None:##
            if not isinstance(data, np.ndarray):
                raise TypeError('{}은 지원하지 않습니다.'.format(type(data)))
        self.data = data
        self.grad = None
        self.creator = None
        
x = Variable(None)

In [3]:
x = Variable(np.array(1.0))

In [4]:
x = Variable(0)

TypeError: <class 'int'>은 지원하지 않습니다.

이와 같이,   
ndarray 나 None 이면 OK.   
But, 다른 데이터타입 사용 시, 예외 발생!!  
<br>

넘파이의 독특한 관례 때문에 문제가 발생되는 상황이 있다.

In [5]:
x = np.array(1.0) # 0차원 ndarray
y = x**2 # numpy.float64 즉 scalar type이 되어버린다.
print(type(x), x.ndim)
print(type(y))

<class 'numpy.ndarray'> 0
<class 'numpy.float64'>


이런 문제 상황을 없애기 위해 isscaler를 이용한, as_array와 같은 편의 함수 생성!  
* np.isscalar(x) : x가 scalar type이면 True, 아니라면 False 반환

In [12]:
print(np.isscalar(np.float64(1.0))) 
print(np.isscalar(2.0))
print(np.isscalar(np.array(1.0))) # 0차원 ndarray 
print(np.isscalar(np.array([0,10]))) # 1차원 ndarray

True
True
False
False


In [6]:
def as_array(x):
    """
    x가 scalar type 이면, ndarray 인스턴스로 변환
    """
    if np.isscalar(x): 
        return np.array(x) 

In [15]:
class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y)) # ouput은 항상 ndarray 인스턴스가 되겠네
        ouput.set_creator(self)
        self.input = input
        self.output = output
        return output

# Step10_테스트
        딥러닝 프레임워크의 테스트 방법
* 테스트를 해야,  실수(버그) 예방 가능
* 테스트 자동화해야, 소프트웨어의 품질 유지 가능
        

In [1]:
import unittest
import numpy as np


class Variable:
    def __init__(self, data):
        if data is not None:
            if not isinstance(data, np.ndarray):
                raise TypeError('{} is not supported'.format(type(data)))

        self.data = data
        self.grad = None
        self.creator = None

    def set_creator(self, func):
        self.creator = func

    def backward(self):
        if self.grad is None:
            self.grad = np.ones_like(self.data)

        funcs = [self.creator]
        while funcs:
            f = funcs.pop()
            x, y = f.input, f.output
            x.grad = f.backward(y.grad)

            if x.creator is not None:
                funcs.append(x.creator)


def as_array(x):
    if np.isscalar(x):
        return np.array(x)
    return x


class Function:
    def __call__(self, input):
        x = input.data
        y = self.forward(x)
        output = Variable(as_array(y))
        output.set_creator(self)
        self.input = input
        self.output = output
        return output

    def forward(self, x):
        raise NotImplementedError()

    def backward(self, gy):
        raise NotImplementedError()


class Square(Function):
    def forward(self, x):
        y = x ** 2
        return y

    def backward(self, gy):
        x = self.input.data
        gx = 2 * x * gy
        return gx


def square(x):
    return Square()(x)


def numerical_diff(f, x, eps=1e-4):
    x0 = Variable(x.data - eps)
    x1 = Variable(x.data + eps)
    y0 = f(x0)
    y1 = f(x1)
    return (y1.data - y0.data) / (2 * eps)


class SquareTest(unittest.TestCase):
    def test_forward(self):
        x = Variable(np.array(2.0))
        y = square(x)
        expected = np.array(4.0)
        self.assertEqual(y.data, expected)

    def test_backward(self):
        x = Variable(np.array(3.0))
        y = square(x)
        y.backward()
        expected = np.array(6.0)
        self.assertEqual(x.grad, expected)


    def test_gradient_check(self):
        """
        기울기 확인 : 수치미분의 결과값, 역전파 결과값 비교해, 차이가 크면 역전파 구현에 문제가 있다고 판단하는 검증 기법
        np.allclose(a,b) : ndarray 인스턴스인 a,b의 값이 얼마나 가까운지 판정
        np.allclose(a,b, rtol=1e-05, atol=1e-8) - > abs(a-b)<=(atol+rtol+abs(b))조건 만족시, True 반환
        """
        x = Variable(np.random.rand(1))
        y = square(x)
        y.backward()
        num_grad = numerical_diff(square, x)
        flg = np.allclose(x.grad, num_grad)
        self.assertTrue(flg)
unittest.main()

E
ERROR: C:\Users\USER\AppData\Roaming\jupyter\runtime\kernel-c8b6f247-8e92-4e9b-83db-0c0fb706578b (unittest.loader._FailedTest)
----------------------------------------------------------------------
AttributeError: module '__main__' has no attribute 'C:\Users\USER\AppData\Roaming\jupyter\runtime\kernel-c8b6f247-8e92-4e9b-83db-0c0fb706578b'

----------------------------------------------------------------------
Ran 1 test in 0.002s

FAILED (errors=1)


SystemExit: True

C:\Users\USER\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3465: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


에러뜨면 안되는데 해결해보자.. module main has no attribute